In [1]:
import pickle
import numpy as np
import pandas as pd

### First, merge the two files that you have

Note to **Farnoud**: please skip to later part, there is a subset sample, so you can load easily

In [2]:
with open(r"C:\Users\zmich\OneDrive - Uniwersytet Jagielloński\Szkoła Doktorska\Miscellaneous\final_res_16-09-22.obj", "rb") as file:
    data1 = pickle.load(file)

In [3]:
with open(r"C:\Users\zmich\OneDrive - Uniwersytet Jagielloński\Szkoła Doktorska\Miscellaneous\final_res_03-11-22.obj", "rb") as file:
    data2 = pickle.load(file)

In [4]:
data = data1 + data2

In [5]:
data.sort(key=lambda x: x["settings"]["Batch"] * 5 + x["settings"]["shared_discount"])

In [6]:
data[3]["settings"]

{'Replication': 0,
 'Batch': 0,
 'shared_discount': 0.32,
 'Start_time': Timestamp('2016-01-01 00:00:17'),
 'End_time': Timestamp('2016-01-01 00:29:45'),
 'Demand_size': 198}

optionally save the object

In [39]:
with open(r"some_path\some_name.obj", "wb") as file:
    pickle.dump(data, file)

## Adding long and lat

**data** is the list with all results

To add longitudes and latitudes, we need to load one full dotmap with exmas results

In [7]:
with open(r"C:\Users\zmich\OneDrive - Uniwersytet Jagielloński\Szkoła Doktorska\Miscellaneous\single_full_dotmap_29-11-22.obj", "rb") as file:
    one_dataset = pickle.load(file)[0]

Small sample for the purpose of recreation: **Farnoud starts here** <br>
For **full analysis ommit the lines**

In [2]:
with open("data_storage/one_dataset_cut.obj", "rb") as file:
    one_dataset = pickle.load(file)

In [3]:
with open("data_storage/small_dataset.obj", "rb") as file:
    data = pickle.load(file)

**Start back here**

In [4]:
nodes_geo = pd.DataFrame({"Node": [t[0] for t in one_dataset["G"].nodes._nodes.items()],
                          "x": [t[1]["x"] for t in one_dataset["G"].nodes._nodes.items()],
                          "y": [t[1]["y"] for t in one_dataset["G"].nodes._nodes.items()]})
nodes_geo[:3]

,Node,x,y
0,42421728,-73.960044,40.798048
1,42421731,-73.961474,40.798654
2,42421737,-73.962873,40.799244


In [5]:
def merging_foo(data, nodes_geo):
    new_requests = data["requests"].merge(nodes_geo, left_on="origin", right_on="Node")\
                    .drop(columns="Node").rename(columns={"x": "x_org", "y": "y_org"})
    new_requests = new_requests.merge(nodes_geo, left_on="destination", right_on="Node")\
                    .drop(columns="Node").rename(columns={"x": "x_dest", "y": "y_dest"})
    data["requests"] = new_requests

In [6]:
for dt in data:
    merging_foo(dt, nodes_geo)

In [7]:
data[0]["requests"][:3]

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,999999,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,999999,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,999999,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207


optional:

In [23]:
with open("some_path\some_name2.obj", "wb") as file:
    pickle.dump(data, file)

Either loop similarly to earlier or create a great df as follows

In [8]:
concatenated_data = pd.concat([d["requests"] for d in data])

In [9]:
concatenated_data[:3]

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,999999,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,999999,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,999999,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207


In [10]:
len(concatenated_data)

1038

In [ ]:
========================================================================================

In [11]:
import h3
import folium
from geojson.feature import Feature, FeatureCollection
import json
import osmnx as ox

In [12]:
# add hexagon column to the DataFrame
df = concatenated_data
zoning_level = 9
df['h3_address'] = df.apply(lambda row: h3.geo_to_h3(row.y_org, row.x_org, zoning_level), axis=1)
df

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest,h3_address
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984,892a100d37bffff
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207,892a1008887ffff
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207,892a10088b3ffff
3,id3888279,2016-01-01 00:01:14,2016-01-01 00:05:54,280,42445649,4461990857,0,1067,57,177,...,2,177,2.220000,1,0,-73.982236,40.751569,-73.991099,40.750342,892a100d66fffff
4,id3058472,2016-01-01 00:27:06,2016-01-01 00:37:26,620,42430688,4461990857,0,2133,1609,355,...,179,355,4.442000,1,0,-73.996437,40.763367,-73.991099,40.750342,892a107258fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,id2147108,2016-01-01 00:59:32,2016-01-01 01:07:29,477,42447182,42422000,0,3253,1768,542,...,241,542,6.776500,1,0,-73.946277,40.792247,-73.973602,40.789613,892a1008d3bffff
242,id0469284,2016-01-01 00:59:37,2016-01-01 01:09:48,611,3786901743,42430375,0,2309,1773,384,...,242,384,4.807500,1,0,-73.981261,40.747033,-73.978141,40.763042,892a100d283ffff
243,id2625897,2016-01-01 00:59:45,2016-01-01 01:04:54,309,42446987,42446934,0,1285,1781,214,...,303,213,2.602150,20,1,-73.973272,40.755232,-73.980649,40.745116,892a100d63bffff
244,id3275015,2016-01-01 00:59:47,2016-01-01 01:03:28,221,42437670,42444954,0,357,1783,59,...,244,59,0.742000,1,0,-73.999148,40.754600,-73.996798,40.752735,892a1072533ffff


In [13]:
# Here are some visualization functions to check the hexagons without colorbar.

def visualize_hexagons(hexagons, color="red", w=1, folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=w,color=color,fill= True)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [14]:
# Using the above function you can visualize the hexagons.

hexagons = list(df.h3_address.unique())
visualize_hexagons(hexagons, color="red", w=1, folium_map=None)

In [15]:
# to plot the KPIs you need to groupby the requests by their hexagon address
# according to you purpose you may use groupby().mean() or groupby().sum()

df_new = df.groupby(['h3_address']).mean()
df_new.reset_index(inplace=True)
df_new

C:\Users\shuli\AppData\Local\Temp\ipykernel_13564\2858625705.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_new = df.groupby(['h3_address']).mean()


,h3_address,trip_duration,origin,destination,status,dist,treq,ttrav,VoT,delta,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,892a1008807ffff,336.000000,4.243110e+07,4.242200e+07,0.0,1107.000000,296.000000,184.0,0.0035,30.527473,...,999999.0,36.000000,184.000000,2.304500,1.000000,0.000000,-73.973205,40.795252,-73.970786,40.788432
1,892a1008817ffff,664.000000,4.242872e+07,2.141027e+09,0.0,3058.000000,889.000000,509.0,0.0035,144.652747,...,999999.0,145.750000,526.750000,6.271653,5.750000,0.000000,-73.970480,40.796637,-73.949196,40.807635
2,892a1008823ffff,623.800000,4.243814e+07,1.519297e+08,0.0,3455.600000,397.000000,575.2,0.0035,143.984176,...,999999.0,28.000000,575.200000,7.196600,1.000000,0.000000,-73.970711,40.789552,-73.952734,40.778518
3,892a100882bffff,818.600000,4.242743e+07,9.718008e+07,0.0,4350.600000,973.400000,724.5,0.0035,192.499121,...,999999.0,384.600000,727.100000,8.810341,12.000000,0.700000,-73.977240,40.789725,-73.946606,40.803762
4,892a100882fffff,479.055556,4.244172e+07,2.021835e+08,0.0,3510.055556,830.666667,584.5,0.0035,159.769597,...,999999.0,445.388889,624.500000,7.075957,15.555556,0.555556,-73.974029,40.786776,-73.958915,40.808138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,892a1072cd3ffff,230.000000,4.245773e+07,4.243685e+07,0.0,729.000000,510.000000,121.0,0.0035,34.562637,...,999999.0,36.000000,121.000000,1.517000,1.000000,0.000000,-74.002970,40.734253,-74.001020,40.730246
211,892a1072cd7ffff,667.000000,3.174308e+08,1.064943e+09,0.0,2595.833333,634.666667,432.0,0.0035,95.883516,...,999999.0,221.333333,465.333333,5.343950,10.666667,0.000000,-74.000655,40.731697,-73.986349,40.733669
212,892a1072cdbffff,932.000000,4.243180e+07,4.243918e+07,0.0,2658.000000,1363.000000,443.0,0.0035,125.597802,...,999999.0,182.500000,469.750000,5.515747,5.750000,0.000000,-74.004561,40.734010,-73.979895,40.735004
213,892a1072dd7ffff,566.000000,5.884557e+08,4.244693e+07,0.0,4236.000000,1449.000000,706.0,0.0035,116.373626,...,999999.0,206.000000,706.000000,8.825000,1.000000,0.000000,-73.985215,40.713789,-73.981105,40.744495


In [16]:
# use this function to get the plots with colorbar.
# adjust the bins before plotting. Otherwise you will see an error.
# you may also modify the other features like colors, opacity and line_weight. 

def h3_visualization(df, hex_col, value_col):
    df['lat'] = df[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
    df['lng'] = df[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
    # df.set_index('hex_address', inplace=True)
    df['geom'] = df.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[hex_col], geo_json = True)]}, axis = 1)
    list_features = []
    for i, row in df.iterrows():
            feature = Feature(geometry = row["geom"],
                              id = row[hex_col],
                              properties = {"resolution": 9})
            list_features.append(feature)
    list_features 

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    geojson_hexes

    # cx, cy = G.nodes[centeral_node]['x'], G.nodes[centeral_node]['y']
    # CENTER = [cy, cx]
    CENTER = [h3.h3_to_geo(df.loc[0][hex_col])[0], h3.h3_to_geo(df.loc[0][hex_col])[1]]

    tile = 'cartodbpositron'
    base_map = folium.Map(zoom_start=11,tiles=tile, zoomControl =  False, location=CENTER)
    
    step = (df[value_col].max() - df[value_col].min())/5
    print(df[value_col].min(),'  ', df[value_col].max()+step, '  ', step)
    bins = list(np.arange(df[value_col].min(),df[value_col].max()+step,step))
    # bins = list(np.arange(1252, 7000, 1000))
    # bins = [0,100,200,300,400,500, 600]

    # for colors see https://user-images.githubusercontent.com/17128994/115975254-c2031e00-a56b-11eb-8025-d82d36bfda1d.png
    color_set = ["Spectral", "RdYlGn", "PuBu", "Accent", "OrRd", "Set1", "Set2", "Set3", "BuPu", "Dark2", "RdBu", "Oranges", "BuGn", "PiYG", "YlOrBr", "YlGn", "Pastel2", "RdPu", "Greens", "PRGn", "YlGnBu", "RdYlBu", "Paired", "BrBG", "Purples", "Reds", "Pastel1", "GnBu", "Greys", "RdGy", "YlOrRd", "PuOr", "PuRd", "Blues", "PuBuGn"]
    m = folium.Choropleth(geo_data = geojson_hexes,data = df, columns = [hex_col,value_col],key_on ="feature.id",
            fill_color='Reds', control = False, bins = bins, line_weight=1, legend_name='',
            fill_opacity=0.5, line_opacity=0.2).add_to(base_map)

    return base_map

In [17]:
h3_visualization(df_new, 'h3_address', 'dist')

357.0    18267.0    2985.0


In [18]:
# to get a plot for number of request for each hexagon I add a column of ones and 
# then use groupby().sum() But you may use another method

df['number_of_req'] = 1
df_new = df.groupby(['h3_address']).sum()
df_new.reset_index(inplace=True)
df_new

C:\Users\shuli\AppData\Local\Temp\ipykernel_13564\1199476348.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_new = df.groupby(['h3_address']).sum()


,h3_address,trip_duration,origin,destination,status,dist,treq,ttrav,VoT,delta,...,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest,number_of_req
0,892a1008807ffff,336,42431099,42421996,0,1107,296,184,0.0035,30.527473,...,36,184,2.304500,1,0,-73.973205,40.795252,-73.970786,40.788432,1
1,892a1008817ffff,2656,169714892,8564106152,0,12232,3556,2036,0.0140,578.610989,...,583,2107,25.086610,23,0,-295.881920,163.186547,-295.796786,163.230538,4
2,892a1008823ffff,3119,212190702,759648263,0,17278,1985,2876,0.0175,719.920879,...,140,2876,35.983000,5,0,-369.853556,203.947760,-369.763672,203.892592,5
3,892a100882bffff,8186,424274288,971800780,0,43506,9734,7245,0.0350,1924.991209,...,3846,7271,88.103410,120,7,-739.772400,407.897246,-739.466063,408.037624,10
4,892a100882fffff,8623,763950910,3639303753,0,63181,14952,10521,0.0630,2875.852747,...,8017,11241,127.367225,280,10,-1331.532529,734.161970,-1331.260471,734.546492,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,892a1072cd3ffff,920,169830912,169747392,0,2916,2040,484,0.0140,138.250549,...,144,484,6.068000,4,0,-296.011881,162.937013,-296.004080,162.920984,4
211,892a1072cd7ffff,4002,1904585098,6389657495,0,15575,3808,2592,0.0210,575.301099,...,1328,2792,32.063700,64,0,-444.003931,244.390182,-443.918094,244.402012,6
212,892a1072cdbffff,3728,169727216,169756724,0,10632,5452,1772,0.0140,502.391209,...,730,1879,22.062990,23,0,-296.018244,162.936040,-295.919580,162.940014,4
213,892a1072dd7ffff,566,588455700,42446933,0,4236,1449,706,0.0035,116.373626,...,206,706,8.825000,1,0,-73.985215,40.713789,-73.981105,40.744495,1


In [19]:
h3_visualization(df_new, 'h3_address', 'number_of_req')

1    27.4    4.4
